# Transformers 模型量化技术：AWQ

![img](https://huggingface.co/datasets/ybelkada/documentation-images/resolve/main/Thumbnail.png)

在2023年6月，Ji Lin等人发表了论文[AWQ：Activation-aware Weight Quantization for LLM Compression and Acceleration](https://arxiv.org/pdf/2306.00978.pdf)。

这篇论文详细介绍了一种激活感知权重量化算法，可以用于压缩任何基于 Transformer 的语言模型，同时只有微小的性能下降。关于 AWQ 算法的详细介绍，见[MIT Han Song 教授分享](https://hanlab.mit.edu/projects/awq)。

transformers 现在支持两个不同的 AWQ 开源实现库：

- [AutoAWQ](https://github.com/casper-hansen/AutoAWQ)
- [LLM-AWQ](https://github.com/mit-han-lab/llm-awq) 


因为 LLM-AWQ 不支持 Nvidia T4 GPU（课程演示 GPU），所以我们使用 AutoAWQ 库来介绍和演示 AWQ 模型量化技术。

## 量化前模型测试文本生成任务

In [1]:
from transformers import pipeline

model_path = "facebook/opt-125m"

# 使用 GPU 加载原始的 OPT-125m 模型
generator = pipeline('text-generation',
                     model=model_path,
                     device=0,
                     do_sample=True,
                     num_return_sequences=3)

#### 实测GPU显存占用：加载 OPT-125m 模型后

```shell
Sun Dec 24 15:11:33 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   47C    P0              26W /  70W |    635MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
```

In [2]:
generator("The woman worked as a")

[{'generated_text': 'The woman worked as a security guard at the casino. According to court records, she worked as a'},
 {'generated_text': 'The woman worked as a housewife at a private residence in the city, while the man worked in'},
 {'generated_text': 'The woman worked as a restaurant manager until the age of 90 and still works as a cashier at'}]

In [3]:
generator("The man worked as a")

[{'generated_text': "The man worked as a security guard. I don't want to know anyone else's whereabouts, but"},
 {'generated_text': 'The man worked as a mechanic at MCA.\nCool. That must be it.'},
 {'generated_text': 'The man worked as a waitress in a major metropolitan area for the past decade. He was an all'}]

## 使用 AutoAWQ 量化模型

下面我们以 `facebook opt-125m` 模型为例，使用 `AutoAWQ` 库实现的 AWQ 算法实现模型量化。

In [4]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer


quant_path = "models/opt-125m-awq"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

# 加载模型
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

C:\Users\kdgc\anaconda3\envs\transformers\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
# 量化模型
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|█████████████████████████████████████████████████████████████████████████████| 12/12 [01:31<00:00,  7.60s/it]


#### 实测GPU显存使用：量化模型时峰值达到将近 4GB

```shell
Sun Dec 24 15:12:50 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   48C    P0              32W /  70W |    3703MiB / 15360MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
```

In [6]:
quant_config

{'zero_point': True, 'q_group_size': 128, 'w_bit': 4, 'version': 'GEMM'}

#### Transformers 兼容性配置

为了使`quant_config` 与 transformers 兼容，我们需要修改配置文件：`使用 Transformers.AwqConfig 来实例化量化模型配置`

In [7]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config

In [8]:
# 保存模型权重
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)  # 保存分词器

('models/opt-125m-awq\\tokenizer_config.json',
 'models/opt-125m-awq\\special_tokens_map.json',
 'models/opt-125m-awq\\vocab.json',
 'models/opt-125m-awq\\merges.txt',
 'models/opt-125m-awq\\added_tokens.json',
 'models/opt-125m-awq\\tokenizer.json')

### 使用 GPU 加载量化模型

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(quant_path)
model = AutoModelForCausalLM.from_pretrained(quant_path, device_map="cuda").to(0)

In [10]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(0)

    out = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(out[0], skip_special_tokens=True)


In [11]:
result = generate_text("Merry Christmas! I'm glad to")
print(result)

Merry Christmas! I'm glad to hear you're doing good! Have a fantastic day!
Well you know how much I love being a Christmas person for you! Thanks!


In [12]:
result = generate_text("The woman worked as a")
print(result)

The woman worked as a model at a local hospital, the attorney representing the company told the AP during a phone interview Thursday.

The incident occurred near St. Paul's St. Paul Health Center in the early morning hours of the morning the day after the suspect was shot. The nurse claimed she was fired after a discussion with police, according


## Homework：使用 AWQ 算法量化 Facebook OPT-2.7B 模型

Facebook OPT 模型：https://huggingface.co/facebook?search_models=opt

In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path_homework = "facebook/opt-2.7B"
quant_path_homework = "models/opt-2.7B-awq"
quant_config_homework = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

# 加载模型
model_homework = AutoAWQForCausalLM.from_pretrained(model_path_homework, device_map="cuda")
tokenizer_homework = AutoTokenizer.from_pretrained(model_path_homework, trust_remote_code=True)

C:\Users\kdgc\anaconda3\envs\transformers\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# 量化模型
model_homework.quantize(tokenizer_homework, quant_config=quant_config_homework)

Repo card metadata block was not found. Setting CardData to empty.
AWQ:   0%|                                                                                      | 0/32 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 960.00 MiB. GPU 

In [ ]:
quant_config_homework

In [ ]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config_homework = AwqConfig(
    bits=quant_config_homework["w_bit"],
    group_size=quant_config_homework["q_group_size"],
    zero_point=quant_config_homework["zero_point"],
    version=quant_config_homework["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model_homework.model.config.quantization_config = quantization_config_homework

In [ ]:
# 保存模型权重
model_homework.save_quantized(quant_path_homework)
tokenizer_homework.save_pretrained(quant_path_homework)  # 保存分词器

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer_homework = AutoTokenizer.from_pretrained(quant_path_homework)
model_homework = AutoModelForCausalLM.from_pretrained(quant_path_homework, device_map="cuda").to(0)

In [ ]:
def generate_text_homework(text):
    inputs = tokenizer_homework(text, return_tensors="pt").to(0)

    out = model_homework.generate(**inputs, max_new_tokens=64)
    return tokenizer_homework.decode(out[0], skip_special_tokens=True)


In [ ]:
result = generate_text_homework("Merry Christmas! I'm glad to")
print(result)

In [ ]:
result = generate_text_homework("The woman worked as a")
print(result)